# LoRA fine-tuning IBM Granite 3.3 with LLaMA Factory for Legal Tasks

In this notebook, we demonstrate how to LoRA (Low-Rank Adaptation) fine-tune the `ibm-granite/granite-3.3-2b-instruct` model using LLaMA Factory on a legal dataset to teach the model about the specific legal domain.

Our notebook is based off of the [notebook example](https://colab.research.google.com/drive/1eRTPn37ltBbYsISy9Aw2NuI2Aq5CQrD9?usp=sharing) from the [LLaMA-Factory repo](https://github.com/hiyouga/LLaMA-Factory/tree/main).

LoRA is an efficient method of fine tuning which reduces the number of trainable parameters which increases the speed and resource usage of training while retaining the same quality. In addition, since the pre-trained weights are kept frozen, the output adapter is lightweight and portable.

[LLaMa Factory](https://github.com/hiyouga/LLaMA-Factory) is an LLM training and fine tuning low/no-code platform that allows users with minimal AI skills to tune LLMs on custom datasets, evaluate performance, and serve models. It has both a web UI and CLI that is user-friendly and supports over 100 LLMs. The platform supports datasets in Alpaca and ShareGPT formats.

The purpose of this notebook is to showcase LoRA tuning of the Granite model to a new domain using this tool. To do this we will walk through the steps:

1. Install necessary dependencies.
2. Download, process, and save the dataset to the format needed for Llama Factory.
3. Set up the training configuration and run tuning with the LLaMA-Factory CLI.
4. Run inference against the base model.
5. Compare inference result against the LoRA tuned model.

By the end, we'll have a model that has learned to interpret legal concepts!

## Google Colab Notes
<p><b> THIS NOTEBOOK REQUIRES A CUDA-ENABLED GPU (NVIDIA GPU) TO RUN. </b></p>
<p>It is designed to work with the free level of GPU available from Google Colab, the T4 GPU. You should not need to pay for GPU time to run this simple tuning demo.</p>
<p>When running in Google Colab, the runtime type should default to a T4 GPU.</p>
<p>If you want to fine-tune using larger datasets, you will need a machine or runtime with a large GPU to perform tuning.</p>
<p>Your local computer can't run this notebook without a CUDA-enabled GPU.</p>
<p><b>Troubleshooting:</b></p>
<p><ul>
<li><b>Verify runtime type:</b> Under the "Runtime" menu, select "Change runtime type." The "Hardware accelerator" field must be set to T4 GPU.</li>
<li><b>Verify runtime is connected:</b> In the top right, you should see the connection status for the T4 runtime. You should see a green checkmark, next to "T4", and "RAM Disk.".</li>
<li><b>Verify T4 GPU is only connected to one Colab session:</b> If you see the word "Connecting" in the top right and it doesn't seem to be resolving, click the arrow dropdown next to it and choose "Manage sessions". If there is an active session already (say, from another run of the notebook in a different browser window, or from using another notebook), you will need to disconnect the session. Click "Terminate other sessions" to do so.</li>
</ul>
</p>

## Install Dependencies

In [ ]:
!pip install -q llamafactory 2>/dev/null
# pandas and numpy are needed to format the dataset
!pip install -q --upgrade numpy 2>/dev/null
!pip install -q --upgrade pandas 2>/dev/null
print("Successfully installed required dependencies for the tutorial!")

### Check GPU environment

A GPU is required to run training with LLaMa-Factory.

In [ ]:
import torch

try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("No GPU found, please set up a GPU before using LLaMA Factory.")

In [ ]:
# Import libraries
import pandas as pd
import json
import yaml

## Download and Process Legal Dataset

For this demonstration, we will use the [`legal-llama-instruction1`](https://huggingface.co/datasets/dliu1/legal-llama-instruction1) dataset from Hugging Face. This dataset contains domain-specific question-and-answer pairs related to the legal field.

Granite 3.3 has been designed to comprehend vast, general-purpose text and to learn through fine-tuning, both of which will be run with LLaMaFactory. Granite models can be efficiently fine-tuned even with limited computing resources.

LLaMaFactory requires the dataset to be preformatted in Alpaca or ShareGPT formats. Thus, we reformat the question and answer fields of the original legal dataset to contain instruction, input, and output fields as per [Alpaca format]((https://github.com/hiyouga/LLaMA-Factory/tree/main/data#alpaca-format)).

Note: You may get a request or error to fetch `HF_TOKEN`. This is a [token from Hugging Face](https://huggingface.co/docs/hub/en/security-tokens) but authentication is optional to access public models or datasets. Because the dataset is public, `HF_TOKEN` is not required to access the dataset. If you would like to add the token to your Colab environment,  you can add the secret following [these instructions](https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0).

In [ ]:
!mkdir -p data
# Download Legal Dataset
df = pd.read_json("hf://datasets/dliu1/legal-llama-instruction1/Instruction_Set.jsonl", lines=True)

# Format Legal Dataset to Alpaca Format
formatted_data = [
    {
        "instruction": row["question"],
        "input": "",
        "output": row["answer"]
    }
    for _, row in df.iterrows()
]

# view line of dataset
print(formatted_data[0])
print("Successfully downloaded and formatted dataset.")

# output formatted legal dataset
with open("data/legal.json", "w", encoding="utf-8") as f:
  json.dump(formatted_data, f, indent=2, ensure_ascii=False)

Llama Factory utilizes a specific file to understand how to load datasets for training. This file must exist at path `data/dataset_info.json`. Thus, we must create a `dataset_info.json` file that includes the path to the new formatted legal dataset we created in order for the Llama Factory CLI to access the dataset. For details on the `dataset_info.json` file see the [documentation](https://github.com/hiyouga/LLaMA-Factory/blob/v0.9.2/data/README.md). Within the Llama Factory repository there are datasets available to use, however, since we are using our own custom dataset we must add our dataset to the JSON file.

In [ ]:
# "legal" will be the identifier for the dataset 
# which points to the local file that contains the dataset
dataset_info = {
  "legal": {
    "file_name": "legal.json",
  }
}

# Create dataset_info.json with legal dataset so can reference with llama factory
with open("data/dataset_info.json", "w", encoding="utf-8") as f:
  json.dump(dataset_info, f, indent=2, ensure_ascii=False)

## LoRA fine-tune model with LLaMa Factory via Command Line

We will be running Supervised Fine-Tuning (SFT) on our legal dataset, targeting all of the linear layers for the LoRA adapter. We set up the training configurations and then write the configs to a YAML file that LLaMa-Factory uses to run training.

See the [docs](https://llamafactory.readthedocs.io/zh-cn/latest/getting_started/sft.html) for more details on the configurations you can set in Llama factory, including other fine tuning techniques and [tuning algorithms](https://llamafactory.readthedocs.io/zh-cn/latest/advanced/adapters.html#lora).

<b>Note: Running this cell loads and fine-tunes the model. This will take about ~8 minutes.</b>

In [ ]:
# setup training configurations
args = dict(
  stage="sft",                                                      # do supervised fine-tuning
  do_train=True,
  model_name_or_path="ibm-granite/granite-3.3-2b-instruct",   # use IBM Granite 3.3 2b instruct model
  dataset="legal",                                                  # use legal datasets we created
  template="granite3",                                              # use granite3 prompt template
  finetuning_type="lora",                                           # use LoRA adapters to save memory
  lora_target="all",                                                # attach LoRA adapters to all linear layers
  loraplus_lr_ratio=16.0,                                           # use LoRA+ algorithm with lambda=16.0
  output_dir="granite3_lora",                                       # the path to save LoRA adapters
  per_device_train_batch_size=4,                                    # the batch size
  gradient_accumulation_steps=2,                                    # the gradient accumulation steps
  learning_rate=1e-4,                                               # the learning rate
  num_train_epochs=3.0,                                             # the epochs of training
  max_samples=500,                                                  # use 500 examples in each dataset
  fp16=True,                                                        # use float16 mixed precision training
  report_to="none",                                                 # disable wandb logging
)

# create training config file to run with llama factory
with open("train_granite3_lora.yaml", "w", encoding="utf-8") as file:
  yaml.dump(args, file, indent=2)

# run training
!llamafactory-cli train train_granite3_lora.yaml

## Infer with the Granite Base Model

We will run inference backed by [HuggingFace generation](https://huggingface.co/docs/transformers/en/main_classes/text_generation), which provides a `model.generate()` method for text generation using PyTorch. With a simple LLaMA-Factory CLI command, we will load the base model and start a chatbot for inference. This will create an ongoing chat stream where the user can input text to run against the model and the response will be streamed back. Once again, we create a YAML file with inference configurations that LLaMa-Factory will use.

Let's try asking the base model a legal question pulled from the dataset. We expect the base model to fail to use legal concepts and vocabulary, since it is a general-purpose model trained on large, diverse datasets. We will compare the results by running against the base model and the LoRA tuned adapter and asking the same question to see how tuning with the legal dataset allowed the model to better understand and answer legal questions.

Let's ask the chatbot the question: `What does assessing unit mean?`

Note: The input typed into the prompt box created from the chat service will be hidden if running in Google Colab as Google Colab hides user input in case a secret is being passed. However, the chat service will print out the input provided so you can see what you inputted.

That chat stream is ongoing so you will have to stop the code block in order to continue.

In [ ]:
# setup inference configurations
args = dict(
  model_name_or_path="ibm-granite/granite-3.3-2b-instruct", # use IBM Granite 3.3 2b instruct model
  template="granite3",                                            # set to the same one used in training, template for constructing prompts
  infer_backend="huggingface"                                     # choices： [huggingface, vllm]
)

# create inference config file to run with llama factory
with open("inference_config.yaml", "w", encoding="utf-8") as file:
  yaml.dump(args, file, indent=2)

# run inference chatbot
!llamafactory-cli chat inference_config.yaml

### Sample output from base model:

- **User**: what does assessing unit mean?
- **Response**: Assessing a unit typically refers to the process of evaluating or measuring the performance, effectiveness, or quality of a specific part or component within a larger system, organization, or structure. This could apply to various contexts such as educational units, business departments, or even individual units in a physical system. The goal is to identify strengths, weaknesses, opportunities for improvement, and potential risks or challenges.

The correct response from the dataset is:

```
question: What does assessing unit mean?
answer: Assessing unit means a city, town, or county with the power to assess real property, unless the city, town or county is part of a consolidated assessing unit; a consolidated assessing unit; or a village as provided in section fourteen hundred two of this chapter.
```

As you can see the answer provided by the base model alone talks about an assessing unit in general terms, as a process that evaluates things. There is no legal information in the answer provided.

## Infer with the LoRA fine-tuned adapter

Now let's use our LoRA tuned adapter with the base model for the chatbot for inference. Since the model was trained on the legal domain, the model's legal knowledge and vocabulary is enhanced. We are pulling the question directly from the legal dataset that we used to tune the model to see how tuning has improved the model's ability to understand legal concepts.

Try again with the question: `What does assessing unit mean?`

In [ ]:
# setup inference configurations
args = dict(
  model_name_or_path="ibm-granite/granite-3.3-2b-instruct", # use IBM Granite 3.3 2b instruct model
  adapter_name_or_path="granite3_lora",                           # load the saved LoRA adapters
  template="granite3",                                            # set to the same one used in training, template for constructing prompts
  finetuning_type="lora",                                         # which fine-tuning technique used in training
  infer_backend="huggingface"                                     # choices： [huggingface, vllm]
)

# create inference config file to run with llama factory
with open("inference_config.yaml", "w", encoding="utf-8") as file:
  yaml.dump(args, file, indent=2)

# run inference chatbot
!llamafactory-cli chat inference_config.yaml

### Sample output from finetuned model:

- **User**: what does assessing unit mean?
- **Response**: Assessing unit means a city, town, or county with the power to assess real property for taxation purposes, or a special district with the power to assess real property for taxation purposes.

The response is much closer to the correct response from the dataset. Instead of defining assessing unit as a process of evaluation, the LoRA tuned model leverages its newly added legal knowledge to describe the unit as a city, town, or county. Instead of describing what is assessed as performance, effectiveness or quality, the tuned model knows that assessing units assess property.

## Summary

In this tutorial you LoRA fine-tuned the IBM Granite-3.3-2b-Instruct model on the legal domain, in order to create an adapter that when used with the base model, can understand and interpret legal concepts. You saw Granite 3.3's capacity to learn even with a small model and limited samples from the dataset.

Try out the notebook with your own dataset to see the power of fine tuning Granite models on new tasks and domains. Let us know what you try!